In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import pandas_datareader as data
start  ='2010-01-01'
end ='2019-12-31'
df=data.DataReader('AAPL','yahoo',start,end)
df =df.reset_index()
df=df.drop(['Date','Adj Close'],axis=1)
ma100=df.Close.rolling(100).mean()
ma200=df.Close.rolling(200).mean()
data_training=pd.DataFrame(df['Close'][0:int(len(df)*0.70)])
data_testing=pd.DataFrame(df['Close'][int(len(df)*0.70):int(len(df))])
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
data_training_array=scaler.fit_transform(data_training)
data_training_array
x_train=[]
y_train=[]
for i in range (100,data_training_array.shape[0]):
    x_train.append(data_training_array[i-100:i])
    y_train.append(data_training_array[i,0])
    
    
x_train,y_train=np.array(x_train), np.array(y_train)
from keras.layers import Dense,Dropout,LSTM
from keras.models import Sequential
model=Sequential()
model.add(LSTM(units=50,activation='relu',return_sequences=True,
                              input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))



model.add(LSTM(units=60,activation='relu',return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(units=80,activation='relu',return_sequences=True))
model.add(Dropout(0.4))



model.add(LSTM(units=120,activation='relu'))
model.add(Dropout(0.5))


model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(x_train,y_train,epochs=50)


Epoch 1/50
52/52 [==============================] - 20s 278ms/step - loss: 0.0556
Epoch 2/50
33/52 [==================>...........] - ETA: 5s - loss: 0.0127

In [ ]:
model.save('keras_model.h5')


In [ ]:
past_100_days=data_training.tail(100)

In [ ]:
final_df =past_100_days.append(data_testing,ignore_index=True)

In [ ]:
input_data=scaler.fit_transform(final_df)

In [ ]:
x_test=[]
y_test=[]
for i in range (100,input_data.shape[0]):
    x_test.append(input_data[i-100:i])
    y_test.append(input_data[i,0])

In [ ]:
x_test ,y_test = np.array(x_test),np.array(y_test)

In [ ]:
y_predicated=model.predict(x_test)


In [ ]:
scaler.scale_


In [ ]:
scale_factor=1/0.02099517
y_predicated=y_predicated*scale_factor
y_test=y_test*scale_factor

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_test, 'b', lablel='Original Price')
plt.plot(y_predicated, 'r', label='predicated price')
plt.xlabel('time')
plt.ylabel('price')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_test, 'b', label='Original Price')
plt.plot(y_predicated, 'r', label='predicated price')
plt.xlabel('time')
plt.ylabel('price')
plt.legend()
plt.show()

In [ ]:
y_predicated
